In [ ]:
import cdsapi
import xarray as xr
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
import nexradaws
import os
import sys
%matplotlib inline

In [ ]:
c = cdsapi.Client()

In [ ]:
def make_a_request(year, month, day):
    request ={'product_type':'reanalysis',
            'format':'netcdf',
            'variable':[
                'divergence','fraction_of_cloud_cover','geopotential',
                'potential_vorticity','relative_humidity',
                'specific_cloud_ice_water_content','specific_cloud_liquid_water_content','specific_humidity',
                'specific_rain_water_content','specific_snow_water_content','temperature',
                'u_component_of_wind','v_component_of_wind','vertical_velocity',
                'vorticity'
            ],
            'pressure_level':[
                '1','2','3',
                '5','7','10',
                '20','30','50',
                '70','100','125',
                '150','175','200',
                '225','250','300',
                '350','400','450',
                '500','550','600',
                '650','700','750',
                '775','800','825',
                '850','875','900',
                '925','950','975',
                '1000'
            ],
            'year': year,
            'month': month,
            'day': day,
            'area': "37.00/-94.00/24.00/-75.00",
            'time':[
                '00:00','01:00','02:00',
                '03:00','04:00','05:00',
                '06:00','07:00','08:00',
                '09:00','10:00','11:00',
                '12:00','13:00','14:00',
                '15:00','16:00','17:00',
                '18:00','19:00','20:00',
                '21:00','22:00','23:00'
            ]
        }
    
    return request


In [ ]:
myr = make_a_request('2019', '01', '01')

In [ ]:
c.retrieve("reanalysis-era5-pressure-levels", myr, "/data/era5/output.nc")

Spatial extent: the keyword area allows to download a very specific subset. The definition is N/W/S/E in degrees longitude and latitude (see API manual). Negative values correspond to S and W. In the example above a domain over Europe.

In [ ]:
ds = xr.open_dataset('/data/era5/output.nc')

In [ ]:
ds

In [ ]:
ds.t[0,0,:,:].plot()

In [ ]:
fme = plt.figure(figsize=[15,8])
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
ds.t.sel(time=0, level=1000, method='nearest').plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree());
ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')


In [ ]:
fme = plt.figure(figsize=[15,8])
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=0.0))
ds.clwc.sel(time=0, level=800, method='nearest').plot.pcolormesh(ax=ax, transform=ccrs.PlateCarree());
ax.coastlines()
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')


In [ ]:
def get_month(year, month, odir):
    c = cdsapi.Client()
    conn = nexradaws.NexradAwsInterface()
    days = conn.get_avail_days(year, month)
    fls = []
    for day in days:
        print('doing ', day)
        # stage to temp dir
        fname = 'era5_seusa' + year + month + day + '.nc'
        outpath = os.path.join(odir, 'era5', year, month)
        try:
            os.makedirs(outpath)
        except FileExistsError:
            pass  # directory exists
        
        myreq = make_a_request(year, month, day)
        
        c.retrieve("reanalysis-era5-pressure-levels", 
                   myreq, os.path.join(outpath, fname))
        fls.append(fname)

    return fls


In [ ]:
res = get_month('2019', '01', '/data/era5/')